<a href="https://colab.research.google.com/github/Hidayathamir/Bot-Telegram-ITK-Prosedur/blob/main/Bot_Telegram_ITK_Prosedur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot

In [ ]:
from telegram.ext import (Updater, ConversationHandler, CommandHandler, MessageHandler, Filters)
import pandas as pd
import numpy as np

data = pd.read_csv('prosedur.csv', sep=';')

In [ ]:
updater = Updater('Your Bot Token', use_context=True)
dp = updater.dispatcher
# ------------------------------------------------------------ #


def start(update, context):
  text = 'Hallo, silahkan lihat /menu yang tersedia.'
  update.message.reply_text(text)
  return ConversationHandler.END

start_handler = CommandHandler('start', start)
dp.add_handler(start_handler)
# ------------------------------------------------------------ #


def menu(update, context):
  text = 'Berikut adalah menu yang tersedia.\n'
  text += '1. /prosedur - Menampilkan seluruh prosedur yang tersedia.\n'
  text += '2. /cariprosedur - Cari prosedur sesuai keyword.'
  update.message.reply_text(text)
  return ConversationHandler.END

menu_handler = CommandHandler('menu', menu)
dp.add_handler(menu_handler)
# ------------------------------------------------------------ #


def prosedur(update, context):
  text = 'Berikut adalah prosedur yang tersedia.\n'
  no = 1
  for row in data.itertuples():
    text += str(no) + '. ' + row.nama_prosedur + ' /prosedur_' + str(row.id) + '\n'
    no += 1
  update.message.reply_text(text)
  return ConversationHandler.END

prosedur_handler = CommandHandler('prosedur', prosedur)
dp.add_handler(prosedur_handler)
# ------------------------------------------------------------ #


QUERY = range(1)
def cariprosedur(update, context):
  update.message.reply_text('Masukkan judul prosedur yang dicari.')
  return QUERY

def hasilcariprosedur(update, context):
  query = update.message.text.strip().lower()

  datum = data[data['nama_prosedur'].str.lower().str.contains(query)].copy()
  if datum.empty:
    text = 'Tidak ditemukan pencarian dengan keyword : ' + query
  else:
    text = 'Berikut adalah hasil pencarian dengan keyword : ' + query + '.\n\n'
    no = 1
    for row in datum.itertuples():
      text += str(no) + '. ' + row.nama_prosedur + ' /prosedur_' + str(row.id) + '\n'
      no += 1
  update.message.reply_text(text)
  return ConversationHandler.END

obrolancariprosedur = ConversationHandler(
    entry_points=[CommandHandler('cariprosedur', cariprosedur)],
    states={QUERY: [MessageHandler(Filters.text, hasilcariprosedur)]},
    fallbacks=[]
)
dp.add_handler(obrolancariprosedur)
# ------------------------------------------------------------ #


def unknown(update, context):
  query = update.message.text.strip().lower()
  if 'prosedur_' in query:
    id = query.replace('/prosedur_', '').strip()
    text = get_langkah_prosedur(id)
    update.message.reply_text(text)

def get_langkah_prosedur(id):
  id = int(id)
  text = 'Tidak ada prosedur dengan id ' + str(id)
  datum = None
  try:
    datum = data[data.id.eq(id)].iloc[0].copy()
  except:
    pass
  if not (datum is None):
    text = 'Berikut adalah langkah prosedur ' + datum.nama_prosedur + '\n' + datum.prosedur
  return text

unknown_handler = MessageHandler(Filters.command, unknown)
dp.add_handler(unknown_handler)
# ------------------------------------------------------------ #


updater.start_polling()
updater.idle()